In [1]:
import pandas as pd
import numpy as np
import time
import clients
import metrics
import ast

from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

In [20]:
import random

def parse_df(file : str) -> pd.DataFrame:
    df = pd.DataFrame(columns=['Id', 'Complex_word', 'Text', 'Candidates'])
    
    with open(file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            line = line.split('\t')
            
            text = line[0]
            complex_word = line[1]
            
            # the rest of the line is the candidates, which will be put in dictionaries. These dictionaries will have the candidate as key and the number of times it appearas as value
            candidates = {}
            for candidate in line[2:]:
                candidate = candidate.strip()
                if candidate in candidates:
                    candidates[candidate] += 1
                else:
                    candidates[candidate] = 1
                    
            # add new row to the dataframe
            new_df = pd.DataFrame([[complex_word, text, candidates]], columns=['Complex_word', 'Text', 'Candidates'])
            df = pd.concat([df, new_df], ignore_index=True)
    
    return df

df = parse_df('../data/tsar2022_es_test_gold.tsv')
df.drop(columns=['Id'], inplace=True)
df.head()
            


,Complex_word,Text,Candidates
0,labrarse,"A comienzos de la década de 1980, se trasladó ...","{'construirse': 4, 'trabajar': 1, 'ganarse': 2..."
1,tertulias,A lo largo de sus más de veinte años de experi...,"{'conversaciones': 2, 'debates': 4, 'reunion':..."
2,propiciado,A pesar de las pocas bajas (menos de 500 en to...,"{'favorecido': 3, 'desencadenado': 3, 'favorid..."
3,decadencia,A pesar de las pocas bajas (menos de 500 en to...,"{'caída': 11, 'decaimiento': 1, 'abatimiento':..."
4,caseríos,Al igual que otros municipios cercanos a Toled...,"{'casas': 5, 'casales': 1, 'trabajos': 1, 'pue..."


## Exploratory Data Analysis

EDA (or Exploratory Data Analysis) is the process of examining and understanding the intrinsics of a dataset in order to obtain the most useful information from it as possible. This process is crucial to understand the data and to make sure that the data is clean and ready to be used in a machine learning model. In this step, it's very useful to apply some statistical techniques and visualizations to understand what we're working with a bit better.

However, in this notebook, the data in question is not numerical, but textual. This means that we can't use the usual methods to analyse data, such as histograms, scatter plots, etc. Instead, Natural Language Processing (NLP) techniques will be used to extract the most important information.

### Basic Dataset Information

#### Dataset Head

#### Dataset Shape

#### Dataset Structure

#### Looking for Null Values

### Text Statistics

#### Word Frecuency Analysis

#### N-Gram Exploration

#### Wordcloud (Without Stopwords)

### Specific Domain Analysis and Metrics

A number of specific metrics to understand a sentence's complexity and readability have been developed over the years. In this notebook, we will use the following metrics, taken from this reference [paper](https://www.researchgate.net/publication/348660072_Automated_Readability_Assessment_for_Spanish_e-Government_Information) on readability assessment for Spanish e-Government information, and the follwing website, [Legible.es](https://legible.es/):
- [μ legibility](https://legible.es/blog/legibilidad-mu/)


#### μ legibility

The readability index μ is a readability index based on the length of words and the mean and variance of the number of letters per word. It is calculated as follows:

$$ μ = \frac{n}{n-1} \times \frac{\bar{x}}{\sigma^2} \times 100 $$

Where:
- $ n $: number of words in the text
- $ \bar{x} $: the mean number of letters per word 
- $ \sigma^2 $: the variance of letters per word


In [21]:
def get_number_of_words(text : str) -> float:
    return len(text.split())

def get_average_number_of_characters(text : str) -> float:
    return sum([len(word) for word in text.split()]) / get_number_of_words(text)

def get_variance_of_letters_per_word(text : str) -> float:
    average = get_average_number_of_characters(text)
    number_of_words = get_number_of_words(text)
    variance = sum([(len(word) - average) ** 2 for word in text.split()]) / number_of_words
    
    return variance

def get_mu_legibility(text : str) -> float:
    n = get_number_of_words(text)
    x = get_average_number_of_characters(text)
    sigma = get_variance_of_letters_per_word(text)
    
    return (n/(n-1)) * (x / sigma) * 100

# add new columns to the dataframe
df['Mu_legibilituy'] = df['Text'].apply(get_mu_legibility)
df.head()



,Complex_word,Text,Candidates,Mu_legibilituy
0,labrarse,"A comienzos de la década de 1980, se trasladó ...","{'construirse': 4, 'trabajar': 1, 'ganarse': 2...",47.048815
1,tertulias,A lo largo de sus más de veinte años de experi...,"{'conversaciones': 2, 'debates': 4, 'reunion':...",38.524947
2,propiciado,A pesar de las pocas bajas (menos de 500 en to...,"{'favorecido': 3, 'desencadenado': 3, 'favorid...",57.156977
3,decadencia,A pesar de las pocas bajas (menos de 500 en to...,"{'caída': 11, 'decaimiento': 1, 'abatimiento':...",57.156977
4,caseríos,Al igual que otros municipios cercanos a Toled...,"{'casas': 5, 'casales': 1, 'trabajos': 1, 'pue...",62.283901


## Obtaining inferred candidates

In [22]:

# client = clients.ChatGPTClient()
# start_time = time.time()
# i = 0
# error_list = []
# df['Chatgpt_inferred_candidates'] = None

# for index, row in df.iterrows():
#     i += 1
#     if i % 10 == 0:
#         print(f'Processed {i} rows in {time.time() - start_time} seconds')
    
#     text = row['Text']
#     complex_word = row['Complex_word']
    
#     try:
#         candidates = client.get_response(text, complex_word)
#         candidates = clients.parse_output(candidates)
        
#         candidate_dict = {}
#         for candidate in candidates:
#             if candidate in candidate_dict:
#                 candidate_dict[candidate] += 1
#             else:
#                 candidate_dict[candidate] = 1
        
        
#         df.at[index, 'Chatgpt_inferred_candidates'] = candidate_dict
#     except Exception as e:
#         error_list.append((index, e))
#         df.at[index, 'Inferred_candidates'] = None

# df.to_csv('../data/Chatgpt_inferred_candidates.csv', sep='\t', index=False)


In [12]:
# df = pd.read_csv('../data/gemini_inferred_candidates.csv')
# client = clients.GeminiClient()
# start_time = time.time()
# i = 1
# error_list = []

# for index, row in df.iterrows():
    
#     if i % 10 == 0:
#         print(f'Processed {i} rows in {time.time() - start_time} seconds')
    
#     text = row['Text']
#     complex_word = row['Complex_word']
    
#     if row['Gemini_inferred_candidates'] is np.nan:
#         i += 1
#         try:
            
#             candidates = client.get_response(text, complex_word)
#             print(candidates)
#             candidates = clients.parse_output(candidates)
            
#             if candidates == []:
#                 break
            
#             candidate_dict = {}
#             for candidate in candidates:
#                 if candidate in candidate_dict:
#                     candidate_dict[candidate] += 1
#                 else:
#                     candidate_dict[candidate] = 1
            
            
#             df.at[index, 'Gemini_inferred_candidates'] = candidate_dict
#         except Exception as e:
#             break

# df.to_csv('../data/gemini_inferred_candidates.csv', index=False)


In [4]:

df = pd.read_csv('../data/gemini_inferred_candidates.csv')

client = clients.LLamaAPIClient(model='llama-70b-chat')
start_time = time.time()
i = 0
error_list = []
df['Llama_inferred_candidates'] = None

for index, row in df.iterrows():
    i += 1
    if i % 10 == 0:
        print(f'Processed {i} rows in {time.time() - start_time} seconds')
        
    if row['Llama_inferred_candidates'] is None:
        
        text = row['Text']
        complex_word = row['Complex_word']
        
        try:
            candidates = client.get_response(text, complex_word)
            candidates = clients.parse_output(candidates)
            
            candidate_dict = {}
            for candidate in candidates:
                if candidate in candidate_dict:
                    candidate_dict[candidate] += 1
                else:
                    candidate_dict[candidate] = 1
            
            
            df.at[index, 'Llama_inferred_candidates'] = candidate_dict
        except Exception as e:
            break

df.to_csv('../data/llama_inferred_candidates.csv', index=False)


Processed 10 rows in 26.327110767364502 seconds
Processed 20 rows in 51.61643433570862 seconds
Processed 30 rows in 76.03284049034119 seconds
Processed 40 rows in 100.85530662536621 seconds
Processed 50 rows in 126.50559997558594 seconds
Processed 60 rows in 149.85234022140503 seconds
Processed 70 rows in 175.41528725624084 seconds
Processed 80 rows in 199.04939794540405 seconds
Processed 90 rows in 223.21880531311035 seconds
Processed 100 rows in 248.25677752494812 seconds
Processed 110 rows in 272.57906794548035 seconds
Processed 120 rows in 298.300137758255 seconds
Processed 130 rows in 324.48309111595154 seconds
Processed 140 rows in 349.62877368927 seconds
Processed 150 rows in 375.08426427841187 seconds
Processed 160 rows in 400.0291213989258 seconds
Processed 170 rows in 427.05443048477173 seconds
Processed 180 rows in 452.6492576599121 seconds
Processed 190 rows in 478.52968645095825 seconds
Processed 200 rows in 517.4172170162201 seconds
Processed 210 rows in 554.3306124210358

In [6]:
df = pd.read_csv('../data/llama_inferred_candidates.csv')
df.head()

,Complex_word,Text,Candidates,Mu_legibilituy,Chatgpt_inferred_candidates,Gemini_inferred_candidates,Llama_inferred_candidates
0,labrarse,"A comienzos de la década de 1980, se trasladó ...","{'construirse': 4, 'trabajar': 1, 'ganarse': 2...",47.048815,"{'surgir': 1, 'forjarse': 1, 'crear': 1, 'cons...","{'construir': 1, 'ganar': 1, 'forjar': 1, 'cre...","{'formar': 1, 'crear': 1, 'construir': 1, 'est..."
1,tertulias,A lo largo de sus más de veinte años de experi...,"{'conversaciones': 2, 'debates': 4, 'reunion':...",38.524947,"{'sobremesas': 1, 'charlas': 1, 'conversacione...","{'debates': 1, 'charlas': 1, 'coloquios': 1, '...","{'debates': 1, 'discusiones': 1, 'conversacion..."
2,propiciado,A pesar de las pocas bajas (menos de 500 en to...,"{'favorecido': 3, 'desencadenado': 3, 'favorid...",57.156977,"{'suscitado': 1, 'provocado': 1, 'fomentado': ...","{'causado': 1, 'provocado': 1, 'originado': 1,...","{'fomentado': 1, 'facilitado': 1, 'impulsado':..."
3,decadencia,A pesar de las pocas bajas (menos de 500 en to...,"{'caída': 11, 'decaimiento': 1, 'abatimiento':...",57.156977,"{'declive': 1, 'declinación': 1, 'decadencia':...","{'caída': 1, 'declive': 1, 'ruina': 1, 'deteri...","{'declive': 1, 'caída': 1, 'deterioro': 1, 're..."
4,caseríos,Al igual que otros municipios cercanos a Toled...,"{'casas': 5, 'casales': 1, 'trabajos': 1, 'pue...",62.283901,"{'sitios': 1, 'aldeas': 1, 'pueblos': 1, 'vill...","{'cabañas': 1, 'chozas': 1, 'aldeas': 1, 'gran...","{'ranchos': 1, 'granjas': 1, 'aldeas': 1, 'pue..."


In [7]:

df = pd.read_csv('../data/llama_inferred_candidates.csv')

client = clients.Palm2Client()
start_time = time.time()
i = 0
error_list = []
df['Palm2_inferred_candidates'] = None

for index, row in df.iterrows():
    i += 1
    if i % 10 == 0:
        print(f'Processed {i} rows in {time.time() - start_time} seconds')
        
    if row['Palm2_inferred_candidates'] is None:
        
        text = row['Text']
        complex_word = row['Complex_word']
        
        try:
            candidates = client.get_response(text, complex_word)
            candidates = clients.parse_output(candidates)
            
            candidate_dict = {}
            for candidate in candidates:
                if candidate in candidate_dict:
                    candidate_dict[candidate] += 1
                else:
                    candidate_dict[candidate] = 1
            
            
            df.at[index, 'Palm2_inferred_candidates'] = candidate_dict
        except Exception as e:
            break

df.to_csv('../data/palm2_inferred_candidates.csv', index=False)
df.head()


Processed 10 rows in 6.751493692398071 seconds
Processed 20 rows in 19.8729829788208 seconds
Processed 30 rows in 26.78946876525879 seconds
Processed 40 rows in 32.52844214439392 seconds
Processed 50 rows in 39.13979244232178 seconds
Processed 60 rows in 46.65919613838196 seconds
Processed 70 rows in 52.95186424255371 seconds
Processed 80 rows in 59.29376292228699 seconds
Processed 90 rows in 66.72850775718689 seconds
Processed 100 rows in 73.29246687889099 seconds
Processed 110 rows in 82.2365927696228 seconds
Processed 120 rows in 89.20549154281616 seconds
Processed 130 rows in 95.52249956130981 seconds
Processed 140 rows in 102.96518230438232 seconds
Processed 150 rows in 110.70376777648926 seconds
Processed 160 rows in 123.8163366317749 seconds
Processed 170 rows in 131.6871054172516 seconds
Processed 180 rows in 138.58925247192383 seconds
Processed 190 rows in 145.5297293663025 seconds
Processed 200 rows in 153.14398741722107 seconds
Processed 210 rows in 161.9184455871582 seconds

,Complex_word,Text,Candidates,Mu_legibilituy,Chatgpt_inferred_candidates,Gemini_inferred_candidates,Llama_inferred_candidates,Palm2_inferred_candidates
0,labrarse,"A comienzos de la década de 1980, se trasladó ...","{'construirse': 4, 'trabajar': 1, 'ganarse': 2...",47.048815,"{'surgir': 1, 'forjarse': 1, 'crear': 1, 'cons...","{'construir': 1, 'ganar': 1, 'forjar': 1, 'cre...","{'formar': 1, 'crear': 1, 'construir': 1, 'est...","{'ganar': 1, 'conseguir': 1, 'adquirir': 1, 'o..."
1,tertulias,A lo largo de sus más de veinte años de experi...,"{'conversaciones': 2, 'debates': 4, 'reunion':...",38.524947,"{'sobremesas': 1, 'charlas': 1, 'conversacione...","{'debates': 1, 'charlas': 1, 'coloquios': 1, '...","{'debates': 1, 'discusiones': 1, 'conversacion...","{'charlas': 1, 'conversaciones': 1, 'debates':..."
2,propiciado,A pesar de las pocas bajas (menos de 500 en to...,"{'favorecido': 3, 'desencadenado': 3, 'favorid...",57.156977,"{'suscitado': 1, 'provocado': 1, 'fomentado': ...","{'causado': 1, 'provocado': 1, 'originado': 1,...","{'fomentado': 1, 'facilitado': 1, 'impulsado':...","{'causado': 1, 'producido': 1, 'originado': 1,..."
3,decadencia,A pesar de las pocas bajas (menos de 500 en to...,"{'caída': 11, 'decaimiento': 1, 'abatimiento':...",57.156977,"{'declive': 1, 'declinación': 1, 'decadencia':...","{'caída': 1, 'declive': 1, 'ruina': 1, 'deteri...","{'declive': 1, 'caída': 1, 'deterioro': 1, 're...","{'deterioro': 1, 'destrucción': 1, 'caída': 1,..."
4,caseríos,Al igual que otros municipios cercanos a Toled...,"{'casas': 5, 'casales': 1, 'trabajos': 1, 'pue...",62.283901,"{'sitios': 1, 'aldeas': 1, 'pueblos': 1, 'vill...","{'cabañas': 1, 'chozas': 1, 'aldeas': 1, 'gran...","{'ranchos': 1, 'granjas': 1, 'aldeas': 1, 'pue...","{'casas': 1, 'viviendas': 1, 'chalets': 2, 'ca..."


In [10]:
df = pd.read_csv('../data/palm2_inferred_candidates.csv')

client =MistralClient()

system = '''            
            ### DESCRIPCIÓN DE LA TAREA ### 

            Quiero que actúes como un anotador de datos para una empresa de aprendizaje automático. Tu tarea es identificar una palabra compleja en una oración y proporcionar 5 alternativas más simples para esa palabra que harían el texto más fácil de leer.
            Recuerda que el objetivo de estas anotaciones es hacer que el texto sea más accesible para personas con dificultades de comprensión lectora, por lo que tienes que tener en cuenta el contexto en el que se usa la palabra.
            A continuación, te proporciono una serie de ejemplos para que aprendas lo que tienes que hacer. El input que te llegará será el texto a partir de la línea de entrada y lo que espero es que me devuelvas SOLO el output a partir de "salida", como en los siguientes ejemplos.

            ### EJEMPLOS ### 	
            Ejemplo 1:
            Prompt del usuario: 
                Frase: El rápido zorro marrón salta sobre el perro perezoso.
                Palabra compleja: salta
            Salida del sistema:
                [brinca,bota,rebota,avanza,retoza]
        
            Ejemplo 2:
            Prompt del usuario: 
                Frase: La tormenta causó un apagón en todo el vecindario.
                Palabra compleja: apagón
            Salida del sistema:
                [corte,oscuridad,interrupción,fallo,apagado]

            Ejemplo 3:
            Prompt del usuario: 
                Frase: El niño encontró un tesoro enterrado en la playa.
                Palabra compleja: enterrado
            Salida del sistema:
                [sepultado,oculto,soterrado,escondido,enterrado]

            Ejemplo 4:
            Prompt del usuario: 
                Frase: El detective siguió las pistas hasta resolver el caso.
                Palabra compleja: resolver
            Salida del sistema:
                [solucionar,concluir,determinar,decidir,arreglar]

            Asegúrate de que las alternativas estén encapsuladas dentro de corchetes cuadrados, separadas por comas y sin espacios entre las palabras. Esta última parte es muy importante.
            No debes proporcionar ningún texto adicional. Tu respuesta debe ser SOLO el resultado esperado, el que se encuentra entre corchetes cuadrados. Las alternativas deben estar en español y ser sinónimos de la palabra compleja.
    ''' 



start_time = time.time()
i = 0
error_list = []
df['mixtral_inferred_candidates'] = None

for index, row in df.iterrows():
    i += 1
    if i % 10 == 0:
        print(f'Processed {i} rows in {time.time() - start_time} seconds')
        
    if row['mixtral_inferred_candidates'] is None:
        
        text = row['Text']
        complex_word = row['Complex_word']
        
        messages = [
            ChatMessage(role='system', content=system),
            ChatMessage(role='user', content=clients.create_user_prompt(text, complex_word))
        ]
        
        try:
            response = client.chat(
                model = 'mistral-large-latest',
                messages = messages
            )
            
            candidates = response.choices[0].message.content
            candidates = clients.parse_output(candidates)
            
            candidate_dict = {}
            for candidate in candidates:
                if candidate in candidate_dict:
                    candidate_dict[candidate] += 1
                else:
                    candidate_dict[candidate] = 1
            
            
            df.at[index, 'mixtral_inferred_candidates'] = candidate_dict
        except Exception as e:
            break

df.to_csv('../data/mixtral_inferred_candidates.csv', index=False)
df.head()


Processed 10 rows in 9.327822208404541 seconds
Processed 20 rows in 19.123793840408325 seconds
Processed 30 rows in 28.54219079017639 seconds
Processed 40 rows in 38.45257496833801 seconds
Processed 50 rows in 48.40049409866333 seconds
Processed 60 rows in 58.53792858123779 seconds
Processed 70 rows in 69.0071668624878 seconds
Processed 80 rows in 79.14643907546997 seconds
Processed 90 rows in 89.69579648971558 seconds
Processed 100 rows in 100.21108770370483 seconds
Processed 110 rows in 110.99037051200867 seconds
Processed 120 rows in 120.096186876297 seconds
Processed 130 rows in 130.33312892913818 seconds
Processed 140 rows in 139.50761890411377 seconds
Processed 150 rows in 148.85224437713623 seconds
Processed 160 rows in 158.81453442573547 seconds
Processed 170 rows in 168.57118773460388 seconds
Processed 180 rows in 177.83355355262756 seconds
Processed 190 rows in 191.564626455307 seconds
Processed 200 rows in 200.98009967803955 seconds
Processed 210 rows in 211.20850610733032 s

,Complex_word,Text,Candidates,Mu_legibilituy,Chatgpt_inferred_candidates,Gemini_inferred_candidates,Llama_inferred_candidates,Palm2_inferred_candidates,mixtral_inferred_candidates
0,labrarse,"A comienzos de la década de 1980, se trasladó ...","{'construirse': 4, 'trabajar': 1, 'ganarse': 2...",47.048815,"{'surgir': 1, 'forjarse': 1, 'crear': 1, 'cons...","{'construir': 1, 'ganar': 1, 'forjar': 1, 'cre...","{'formar': 1, 'crear': 1, 'construir': 1, 'est...","{'ganar': 1, 'conseguir': 1, 'adquirir': 1, 'o...","{'construirse': 1, 'forjarse': 1, 'edificarse'..."
1,tertulias,A lo largo de sus más de veinte años de experi...,"{'conversaciones': 2, 'debates': 4, 'reunion':...",38.524947,"{'sobremesas': 1, 'charlas': 1, 'conversacione...","{'debates': 1, 'charlas': 1, 'coloquios': 1, '...","{'debates': 1, 'discusiones': 1, 'conversacion...","{'charlas': 1, 'conversaciones': 1, 'debates':...","{'charlas': 1, 'debates': 1, 'conversaciones':..."
2,propiciado,A pesar de las pocas bajas (menos de 500 en to...,"{'favorecido': 3, 'desencadenado': 3, 'favorid...",57.156977,"{'suscitado': 1, 'provocado': 1, 'fomentado': ...","{'causado': 1, 'provocado': 1, 'originado': 1,...","{'fomentado': 1, 'facilitado': 1, 'impulsado':...","{'causado': 1, 'producido': 1, 'originado': 1,...","{'facilitado': 1, 'promovido': 1, 'favorecido'..."
3,decadencia,A pesar de las pocas bajas (menos de 500 en to...,"{'caída': 11, 'decaimiento': 1, 'abatimiento':...",57.156977,"{'declive': 1, 'declinación': 1, 'decadencia':...","{'caída': 1, 'declive': 1, 'ruina': 1, 'deteri...","{'declive': 1, 'caída': 1, 'deterioro': 1, 're...","{'deterioro': 1, 'destrucción': 1, 'caída': 1,...","{'declive': 1, 'caída': 1, 'decaimiento': 1, '..."
4,caseríos,Al igual que otros municipios cercanos a Toled...,"{'casas': 5, 'casales': 1, 'trabajos': 1, 'pue...",62.283901,"{'sitios': 1, 'aldeas': 1, 'pueblos': 1, 'vill...","{'cabañas': 1, 'chozas': 1, 'aldeas': 1, 'gran...","{'ranchos': 1, 'granjas': 1, 'aldeas': 1, 'pue...","{'casas': 1, 'viviendas': 1, 'chalets': 2, 'ca...","{'viviendas': 1, 'casas': 1, 'hogares': 1, 'mo..."


In [13]:
df.head()

,Complex_word,Text,Candidates,Mu_legibilituy,Chatgpt_inferred_candidates,Gemini_inferred_candidates,Llama_inferred_candidates,Palm2_inferred_candidates,Mixtral_inferred_candidates
0,labrarse,"A comienzos de la década de 1980, se trasladó ...","{'construirse': 4, 'trabajar': 1, 'ganarse': 2...",47.048815,"{'surgir': 1, 'forjarse': 1, 'crear': 1, 'cons...","{'construir': 1, 'ganar': 1, 'forjar': 1, 'cre...","{'formar': 1, 'crear': 1, 'construir': 1, 'est...","{'ganar': 1, 'conseguir': 1, 'adquirir': 1, 'o...","{'construirse': 1, 'forjarse': 1, 'edificarse'..."
1,tertulias,A lo largo de sus más de veinte años de experi...,"{'conversaciones': 2, 'debates': 4, 'reunion':...",38.524947,"{'sobremesas': 1, 'charlas': 1, 'conversacione...","{'debates': 1, 'charlas': 1, 'coloquios': 1, '...","{'debates': 1, 'discusiones': 1, 'conversacion...","{'charlas': 1, 'conversaciones': 1, 'debates':...","{'charlas': 1, 'debates': 1, 'conversaciones':..."
2,propiciado,A pesar de las pocas bajas (menos de 500 en to...,"{'favorecido': 3, 'desencadenado': 3, 'favorid...",57.156977,"{'suscitado': 1, 'provocado': 1, 'fomentado': ...","{'causado': 1, 'provocado': 1, 'originado': 1,...","{'fomentado': 1, 'facilitado': 1, 'impulsado':...","{'causado': 1, 'producido': 1, 'originado': 1,...","{'facilitado': 1, 'promovido': 1, 'favorecido'..."
3,decadencia,A pesar de las pocas bajas (menos de 500 en to...,"{'caída': 11, 'decaimiento': 1, 'abatimiento':...",57.156977,"{'declive': 1, 'declinación': 1, 'decadencia':...","{'caída': 1, 'declive': 1, 'ruina': 1, 'deteri...","{'declive': 1, 'caída': 1, 'deterioro': 1, 're...","{'deterioro': 1, 'destrucción': 1, 'caída': 1,...","{'declive': 1, 'caída': 1, 'decaimiento': 1, '..."
4,caseríos,Al igual que otros municipios cercanos a Toled...,"{'casas': 5, 'casales': 1, 'trabajos': 1, 'pue...",62.283901,"{'sitios': 1, 'aldeas': 1, 'pueblos': 1, 'vill...","{'cabañas': 1, 'chozas': 1, 'aldeas': 1, 'gran...","{'ranchos': 1, 'granjas': 1, 'aldeas': 1, 'pue...","{'casas': 1, 'viviendas': 1, 'chalets': 2, 'ca...","{'viviendas': 1, 'casas': 1, 'hogares': 1, 'mo..."


In [4]:
# combine all the candidates into one column
df = pd.read_csv('../data/mixtral_inferred_candidates.csv'  )

def combine_dict_list(dict_list : list[dict]) -> dict:
    combined_dict = {}
    for d in dict_list:
        for key in d:
            if key in combined_dict:
                combined_dict[key] += d[key]
            else:
                combined_dict[key] = d[key]
    
    return combined_dict

for index, row in df.iterrows():
    candidates = [
        ast.literal_eval(row['Chatgpt_inferred_candidates']), 
        ast.literal_eval(row['Gemini_inferred_candidates']), 
        ast.literal_eval(row['Llama_inferred_candidates']),
        ast.literal_eval(row['Palm2_inferred_candidates']), 
        ast.literal_eval(row['Mixtral_inferred_candidates'])
    ]
    combined_dict = combine_dict_list(candidates)
    
    # sort the dictionary by value
    combined_dict = dict(sorted(combined_dict.items(), key=lambda item: item[1], reverse=True))
    
    df.at[index, 'Total_inferred_candidates'] = str(combined_dict)
    
df.to_csv('../data/total_inferred_candidates.csv', index=False)

